In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_698.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_386.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_858.wav
/kaggle/input/shl-intern-hiring-ass

In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor

In [3]:
train_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv")

In [4]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'zcr': np.mean(librosa.feature.zero_crossing_rate(y).T),
        'rms': np.mean(librosa.feature.rms(y=y).T),
        'centroid': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T),
        'bandwidth': np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr).T),
        'rolloff': np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T),
        'mfcc_mean': np.mean(librosa.feature.mfcc(y=y, sr=sr), axis=1).mean(),
        'mfcc_std': np.std(librosa.feature.mfcc(y=y, sr=sr), axis=1).mean(),
    }
    return features

In [5]:
train_features_list = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    file_path = f"/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/{row['filename']}"
    try:
        feats = extract_features(file_path)
        feats['label'] = row['label']
        feats['filename'] = row['filename']
        train_features_list.append(feats)
    except:
        print("Error in:", row['filename'])

train_features_df = pd.DataFrame(train_features_list)
train_features_df.head()

100%|██████████| 444/444 [05:02<00:00,  1.47it/s]


,zcr,rms,centroid,bandwidth,rolloff,mfcc_mean,mfcc_std,label,filename
0,0.143139,0.015064,1812.681453,1710.677609,3517.925486,-22.437603,19.341022,1.0,audio_1261.wav
1,0.105385,0.037655,1962.232377,1590.935140,3678.718218,-22.819935,18.391357,1.5,audio_942.wav
2,0.132682,0.021168,2126.143258,1593.561943,3834.469016,-22.640741,19.188875,1.5,audio_1110.wav
3,0.094663,0.041441,1296.033816,1256.154774,2438.423522,-18.288782,22.096088,1.5,audio_1024.wav
4,0.136809,0.043016,1601.473150,1444.474832,2949.576511,-14.756093,17.140703,2.0,audio_538.wav


In [6]:
X = train_features_df.drop(columns=['filename', 'label'])
y = train_features_df['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
final_model = CatBoostRegressor(
    learning_rate=0.1,
    l2_leaf_reg=7,
    iterations=700,
    depth=8,
    verbose=0,
    random_state=42
)

final_model.fit(X_train, y_train)

In [8]:
val_preds = final_model.predict(X_val)
rmse = mean_squared_error(y_val, val_preds, squared=False)
r2 = r2_score(y_val, val_preds)

print("Validation RMSE:", rmse)
print("Validation R² Score:", r2)

Validation RMSE: 0.9960602031762226
Validation R² Score: 0.2721733097039226


In [9]:
test_features_list = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    file_path = f"/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/{row['filename']}"
    try:
        feats = extract_features(file_path)
        feats['filename'] = row['filename']
        test_features_list.append(feats)
    except:
        print("Error in:", row['filename'])

test_features_df = pd.DataFrame(test_features_list)

100%|██████████| 195/195 [01:59<00:00,  1.63it/s]


In [10]:
X_test = test_features_df.drop(columns=['filename'])
test_preds = final_model.predict(X_test)

In [11]:
submission = pd.DataFrame({
    'filename': test_features_df['filename'],
    'label': test_preds
})

submission.to_csv("submission.csv", index=False)